<a href="https://colab.research.google.com/github/rraulison/olx/blob/main/scrap_car_olx_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests
!pip install bs4

In [2]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json


headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36",
}

def get_pages(url):
  #faz scrap olx e retorna um json
  response = requests.get(url, headers=headers)
  soup = BeautifulSoup(response.content, "html.parser")

  if response.status_code != 200:
          print(f"A URL {url} não está disponível. O código de status HTTP é {response.status_code}.")
          return None  # Retorna None para indicar que a URL não está disponível

  # Se a página estiver disponível, prossiga com a lógica para criar o DataFrame

  # Find the script tag with id "__NEXT_DATA__"
  script_tag = soup.find('script', {'id': '__NEXT_DATA__'})

  # Extract the text within the script tag
  if script_tag:

    find_this_text = script_tag.get_text(strip=True)
  try:
    # Attempt to parse 'find_this_text' as JSON
    data = json.loads(find_this_text)

    # Now, 'data_as_json' contains the JSON data
    #print(data)
  except json.JSONDecodeError as e:
      print("Error parsing JSON:", e)
  return data

In [75]:
# Lista de URLs diferentes
urls = []
for i in range(1,33):
  urls.append('https://www.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios/estado-ce?cf=5&me=120000&pe=40000&ps=20000&rs=61&o='+str(i))

In [84]:
def geras(df):

  modelo = []
  marca = []
  ano = []
  quilometragem = []
  motor = []
  cambio = []
  direcao = []
  opcionais = []
  tipo = []
  preco = []

  for i in range(len(df['props']['pageProps']['ads'])):
    try:
      for key in df['props']['pageProps']['ads'][i]['properties']:
        if key['name'] == 'vehicle_model':
          modelo.append(key['value'])
        if key['name'] == 'vehicle_brand':
          marca.append(key['value'])
        if key['name'] == 'regdate':
          ano.append(int(key['value']))
        if key['name'] == 'mileage':
          quilometragem.append(int(key['value']))
        if key['name'] == 'motorpower':
          motor.append(key['value'])
        if key['name'] == 'gearbox':
          cambio.append(key['value'])
        if key['name'] == 'car_steering':
          direcao.append(key['value'])
        if key['name'] == 'car_features':
          opcionais.append(key['value'])
        if key['name'] == 'cartype':
          tipo.append(key['value'])
    except: continue
    preco.append(int(df['props']['pageProps']['ads'][i]['price'].replace("R$", "").replace(".","")))
  return [modelo,marca,ano,quilometragem,motor,cambio,direcao,opcionais,preco, tipo]

In [85]:
dfs = []
for page in urls:
  df = pd.DataFrame(geras(get_pages(page))).T
  dfs.append(df)

In [86]:
resultado = pd.concat(dfs, )

In [87]:
resultado

,0,1,2,3,4,5,6,7,8,9
0,FIAT PALIO WAY 1.0 FIRE FLEX 8V 5P,FIAT,2016,66000,1.0,Manual,Mecânica,Air bag,24800,Hatch
1,FORD KA 1.0 SE 12V FLEX 4P MANUAL,FORD,2017,82000,1.0,Manual,Elétrica,"Alarme, Air bag, Sensor de ré, Som, Trava elét...",36900,Hatch
2,FIAT PALIO ATTRA. BEST SELLER 1.0 EVO FLEX 5P,FIAT,2014,89000,1.0,Manual,Hidráulica,"Air bag, Ar condicionado, Trava elétrica, Vidr...",30900,Hatch
3,VOLKSWAGEN UP! MOVE 1.0 TSI TOTAL FLEX 12V 5P,VOLKSWAGEN,2015,99989,1.0,Semi-Automático,Elétrica,"Alarme, Air bag, Ar condicionado, Sensor de ré...",36500,Hatch
4,FORD FIESTA SE STYLE 1.6 16V FLEX MEC. 5P,FORD,2014,89361,1.5,Manual,Elétrica,"Alarme, Air bag, Ar condicionado, Som, Trava e...",36000,Hatch
...,...,...,...,...,...,...,...,...,...,...
8,CHEVROLET MERIVA MAXX 1.4 MPFI 8V ECONOFLEX 5P,CHEVROLET,2012,32000,1.4,Manual,Hidráulica,"Alarme, Air bag, Ar condicionado, Som, Trava e...",38990,Hatch
9,FORD FIESTA 1.0 8V FLEX/CLASS 1.0 8V FLEX 5P,FORD,2011,90000,1.0,Manual,Hidráulica,"Alarme, Air bag, Ar condicionado, Som, Trava e...",23900,Hatch
10,FORD KA 1.0 SE 12V FLEX 4P MANUAL,FORD,2017,65000,1.4,Manual,Hidráulica,"Air bag, Ar condicionado, Som, Trava elétrica,...",38800,Hatch
11,FIAT PALIO ATTRA./ITALIA 1.4 EVO F.FLEX 8V 5P,FIAT,2013,91908,1.6,Manual,Elétrica,"Alarme, Air bag, Ar condicionado, Sensor de ré...",37990,SUV
